# Backend

## Crime Analysis Dashboard App

In [9]:
#Import Libraries
import requests
import pandas as pd
import numpy

## Get Police Force Table

In [ ]:

def get_forces():

    """ 
    
    Defines the function to call the API and retrieve the police forces data table.
    
    Input: None
    
    Output: Dataframe containing all the police forece ids and names.
    
    """

    # Defines the connection to the API
    url = "https://data.police.uk/api/forces"
    response = requests.get(url)

    # Raises exception if connection fails
    if response.status_code != 200:                                 
        raise Exception(f"API error: {response.status_code}")           
    
    # Converts responce to json
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)

    # Rename ID and name columns
    df.rename(columns={"id": "police_force_id", "name": "police_force_name"}, inplace=True)

    # Returns dataframe
    return df

# Defines the police force dataframe
df_forces = get_forces()

print(df_forces.head())

     police_force_id               police_force_name
0  avon-and-somerset  Avon and Somerset Constabulary
1       bedfordshire             Bedfordshire Police
2     cambridgeshire     Cambridgeshire Constabulary
3           cheshire           Cheshire Constabulary
4     city-of-london           City of London Police


## Get Neighbourhood Table

In [11]:

# INPUT FROM USER
police_force_ids = ["bedfordshire", "cambridgeshire"]  

def get_neighbourhood(id):

    """

    Defines the function to call the API and retrieve the neighbourhood data table.

    Input: Police force ID.

    Output: Dataframe containing the neighbouthood id, name and which police force id it belongs to.

    """

    # Defines the connection to the API
    url = f"https://data.police.uk/api/{id}/neighbourhoods"     # Defines the API url 
    response = requests.get(url)                                # Defines the responce after we 'get' the url
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Converts responce to json 
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)

    # Adds police force ID
    df['police_force_id'] = id

    # Renames the ID and name columns
    df.rename(columns={"id": "neighbourhood_id", "name": "neighbourhood_name"}, inplace=True)

    # Returns dataframe
    return df

# Empty list of neighbourhood dataframes 
df_neighbourhoods_list =[]

# Appends each neighbourhood dataframe to the list
for id in police_force_ids:
    df_neighbourhoods_list.append(get_neighbourhood(id))

# Unions the list of dataframes into one
df_neighbourhoods = pd.concat(df_neighbourhoods_list)

print(df_neighbourhoods.sample(5))

         neighbourhood_id              neighbourhood_name police_force_id
14                    NU2          Queens Park and Castle    bedfordshire
2                     BD5               Riseley, Wyboston    bedfordshire
5                     CB3      Leighton Buzzard and Rural    bedfordshire
0     CamCity_City_Centre  Cambridge City Centre and West  cambridgeshire
10  Peterborough_Northern              Peterborough North  cambridgeshire


## Get Neighbourhood Boundaries Table

In [12]:

def get_neighbourhood_boundaries(neighbourhood_id):

    """

    Defines the function to call the API and retrieve the neighbourhood boundaries data table.

    Input: Neighbourhood ID

    Output: Corresponding boundary polygon string
    
    """

    # Finds the input's corresponding police force ID 
    police_force_id = df_neighbourhoods.loc[df_neighbourhoods["neighbourhood_id"] == neighbourhood_id, "police_force_id"].iloc[0]

    # Defines the connection to the API
    url = f"https://data.police.uk/api/{police_force_id}/{neighbourhood_id}/boundary"
    response = requests.get(url)
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Convert responce to json
    data = response.json()

    # Convert to polygon string format
    polygon_str = ":".join(
        f"{float(item['latitude'])},{float(item['longitude'])}"
        for item in data
    )

    # Returns polygon
    return polygon_str

# Makes a copy of the neighbourhoods dataframe
df_neighbourhood_boundaries = df_neighbourhoods.copy()

# Initiates empty boundary list 
boundary_list = []

# Loops through all neighbourhood ID's and appends the boundary polygon to the list
for neighbourhood_id in df_neighbourhood_boundaries["neighbourhood_id"]:
    boundary_list.append(get_neighbourhood_boundaries(neighbourhood_id))

# Defines the boundary list as a new column in the dataframe
df_neighbourhood_boundaries["neighbourhood_boundary"] = boundary_list

print(df_neighbourhood_boundaries.head())

  neighbourhood_id                         neighbourhood_name police_force_id  \
0              BD2                   Bromham, Oakley, Wootton    bedfordshire   
1              BD3           Wilstead, Shortstown, Willington    bedfordshire   
2              BD5                          Riseley, Wyboston    bedfordshire   
3              CB1  Flitwick, Ampthill, Marston and Cranfield    bedfordshire   
4              CB2    Biggleswade, Sandy, Potton and Shefford    bedfordshire   

                              neighbourhood_boundary  
0  52.200075358,-0.548369711:52.200155837,-0.5484...  
1  52.201040284,-0.437150426:52.20106348,-0.43761...  
2  52.322913907,-0.465104578:52.322953267,-0.4653...  
3  52.110251611,-0.591229997:52.11069521,-0.59140...  
4  52.190902794,-0.287653471:52.190925829,-0.2891...  


## Get Street-Level Crimes 

In [13]:

def get_street_level_crimes(neighbourhood_id):

    """
    
    Defines the function to call the API and retrieve the neighbourhood boundaries data table.

    Input: Neighbourhood ID

    Output: Street-level crime dataframe for the neighbourhood

    """

    # Finds the neighbourhoods boundary polygon
    polygon = df_neighbourhood_boundaries.loc[df_neighbourhood_boundaries["neighbourhood_id"] == neighbourhood_id, "neighbourhood_boundary"].iloc[0]

    # Defines the connection to the API
    url = f"https://data.police.uk/api/crimes-street/all-crime?poly={polygon}"
    response = requests.get(url)
    
    # Raises exception if connection fails
    if response.status_code != 200:
        raise Exception(f"API error: {response.status_code}")
    
    # Convert responce to json
    data = response.json()

    # Converts json to dataframe
    df = pd.DataFrame(data)




    return df

    
print(get_street_level_crimes("BD2"))




Exception: API error: 414